The standard RAG workflow

<image src="./images/standard_rag_workflow.png" alt="RAG Workflow" width="600">

-----

Preparing data for retrieval

In [ ]:
# Document Loaders
from langchain_core.document_loaders.base import CSVLoader
csv_loader = CSVLoader(file_path = "path/to/your/file.csv")
documents = csv_loader.load()
print(documents)


# Loading PDF files
from langchain_core.document_loaders.base import PyPDFLoader
pdf_loader = PyPDFLoader(file_path = "path/to/your/file.pdf")
documents = pdf_loader.load()
print(documents)


# Loading html files
from langchain_core.document_loaders.base import UnstructuredHTMLLoader
html_loader = UnstructuredHTMLLoader(file_path = "path/to/your/file.html")
documents = html_loader.load()
first_document = documents[0]

print("Content: ", first_document.page_content)
print("Metadata: ", first_document.metadata)

Perparing data for retreival

In [ ]:
# Chunk Size
#    - Big chunks are slow and difficult to interpret
#    - Small chunks are fast but may lose context

# Chunk Overlap
#    - Overlapping chunks can help maintain context

from langchain_text_splitters import CharacterTextSplitter

text = """Machine learning is a fascinating field.\n\nIt involves algorithms and models that 
can learn from data. These models can then make predictions or decisions without being 
explicitly programmed to perform the task. \nThis capability is increasingly valuable in 
various industries, from finance to healthcare. \n\nThere are many types of machine Learning, 
including supervised, unsupervised, and reinforcement learning. \nEach type has its own strengths and applications. """

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10
)

chunks = text_splitter.split_text(text)
print(chunks)
print([len(chunk) for chunk in chunks])

In [ ]:
# Recursive Text Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    separators=["\n\n", "\n", " ", ""]
)

chunks = splitter.split_text(text)
print(chunks)
print([len(chunk) for chunk in chunks])

Embedding and sotring the chunks

In [ ]:
! pip install sentence-transformers chromadb

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embedding_model = HuggingFaceEmbeddings (
  model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vector_store = Chroma.from_documents(
  documents=chunks, 
  embedding=embedding_model
)

Langchain expression language (LCEL) for RAG
  - It is a declarative syntax for describing chains from prototypes to production
  - It create modular retrieval pipelines which can combine retrieval and generation components together

<image src="./images/lcel.png" alt="RAG Workflow" width="600">

In [ ]:
vector_store = Chroma.from_documents(
  documents=chunks, 
  embedding=embedding_model
)

# Instantiate a retriever
retriever = vector_store.as_retriever(
  search_type="similarity", 
  search_kwargs={"k": 2}
)



# Creating a prompt template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
  Use the following pieces of context to answer the question at the end.
  If you don't know the answer, say that you don't know.
  Context: {context}
  Question: {question}
""")



# Building a lcel retrieval chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context": retriever, "question": RunnablePassthrough() } # RunnblePassthrough allows you to pass the question directly to the chain
    | prompt
    | llm
    | StrOutputParser()
)

result = chain.invoke({"question": "What are the key findings or results presented in the paper?"})
print(result)

-------

Extending the data retrieval with other files

In [ ]:
# Loading markdown files
from langchain_core.document_loaders.base import UnstructuredMarkdownLoader
markdown_loader = UnstructuredMarkdownLoader(file_path="path/to/your/file.md")
markdown_content = markdown_loader.load()
print(markdown_content[0])

In [ ]:
# Loading python files
from langchain_core.document_loaders.base import PythonLoader
python_loader = PythonLoader(file_path="path/to/your/file.py")
python_content = python_loader.load()
print(python_content[0])

In [ ]:
# Splitting the code files
from langchain_text_splitters import Language

python_splitter = RecursiveCharacterTextSplitter.from_language(
  language=Language.PYTHON,
  chunk_size=150,
  chink_overlap=10,
)

chunks = python_splitter.split_documents(python_data)
for i, chunk in enumerate(chunks[:3]):
  print(f"Chunk {i+1}:\n{chunk.page_content}\n")

------

Advanced Splitting Methods

<image src="./images/limitation of splitting.png" alt="RAG Workflow" width="600">

In [ ]:
# Splitting tokens
import tiktoken
from langchain_text_splitters import TokenTextSplitter

example_string = "Mary had a little lamb, it's fleece was white as snow."

encoding = tiktoken.encoding_for_model('gpt-4o-mini')
splitter = TokenTextSplitter(
  encoding_name=encoding.name,
  chunk_size=10, 
  chunk_overlap=2
)

chunks = splitter.split_text(example_string)

for i, chunk in enumerate (chunks) :
  print(f"Chunk {i+1}: nNo. tokens: {len(encoding.encode(chunk))}\n{chunk}\n")

Semantic Splitting

In [ ]:
from langchain_openai import OpenAIEmbeddings 
from langchain_experimental.text_splitter import SemanticChunker

embeddings = OpenAIEmbeddings(api_key="...", model='text-embedding-3-small')

semantic_splitter = SemanticChunker(
    embeddings=embeddings, 
    breakpoint_threshold_type="gradient",
    breakpoint_threshold_amount=0.8
)

chunks = semantic_splitter.split_documents(document)
print(chunks[0])

---------

Optimizing document retrieval
- Dense : Encode chunks as a single vector with non-zero components. Pros: Capture semantic meaning. Cons: Computationally expensive.
- Sparse : Encode using word matching with mostly zero components. Pros: Precise, explainable, rare-word handling Cons: Generalizability

Sparse Retrieval Methods
- TF-IDF : Encodes documents using the words that make the document unique
- BM25 : Helps mitigate high-frequency words from saturating the encoding 

In [5]:
# If not already installed, install the langchain_community package
from langchain_community.retrievers import BM25Retriever

chunks = [
  "Python was created by Guido van Rossum and released in 1991.",
  "Python is a popular language for machine learning (ML).",
  "The PyTorch library is a popular Python library for AI and ML."
]

bm25_retriever = BM25Retriever.from_texts(chunks, k=3)  # K is the number of top results to return
results = bm25_retriever.get_relevant_documents("When was Python created?")
print("Most relevant documents:")
print(results[0].page_content)

Most relevant documents:
Python was created by Guido van Rossum and released in 1991.


In [ ]:
retriever = BM25Retriever.from_documents(
  documents=chunks,
  k=5
)

chain = (
  {"context": retriever, "question": RunnablePassthrough() }
  | prompt
  | llm
  | StrOutputParser()
)

print(chain.invoke({"question": "When was Python created?"}))

-----

Output accuracy: string evaluation

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

query = "What are the main components of RAG architecture?"
predicted_answer = "Training and encoding"
ref_answer = "Retrieval and generation"


prompt_template = """You are an expert professor specialized in grading students' answers to
You are grading the following question: {query}
Here is the real answer: {answer}
You are grading the following predicted answer: {result}
Respond with CORRECT or INCORRECT:
Grade: 
"""

prompt = PromptTemplate(
  input_variables=["query", "answer", "result"], 
  template=prompt_template
)

eval_llm = ChatOpenAI(temperature=0, model="gpt-40-mini", openai_api_key='...')

In [ ]:
from langsmith.evaluation import LangChainStringEvaluator

qa_evaluator = LangChainStringEvaluator(
  "qa",
  config={
    "llm" : eval_llm,
    "prompt" : prompt    
  }
)

score = qa_evaluator.evaluator.evaluate_strings(
  prediction=predicted_answer,
  reference=ref_answer,
  input=query
)

print(f"Score: {score}")
# Score 0 represents INCORRECT, 1 represents CORRECT

----

RAGAS Framework : Faithfulness
- Does the generated output faithfully represent the context?
    - Faithfulness = No. of claims that can be inferred from the context / Total no. of claims in the output 
- Normalized to (0,1) -> 1 = highly relevant

In [ ]:
from langchain_openai import ChatOpenAI, OpenAlEmbeddings 
from ragas.integrations.langchain import EvaluatorChain 
from ragas.metrics import faithfulness

llm = ChatOpenAI(model="gpt-40-mini", api_key="...")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", api_key="...")

faithfulness_chain = EvaluatorChain (
  metric=faithfulness,
  llm=llm,
  embeddings=embeddings
)


eval_result = faithfulness_chain({
"question": "How does the RAG model improve question answering with LLMs?",
"answer": "The RAG model improves question answering by combining the retrieval of documents...",
"contexts": [
  "The RAG model integrates document retrieval with LLMs by first retrieving relevant passages...",
  "By incorporating retrieval mechanisms, RAG leverages external knowledge sources, allowing the...",
]
})

print(eval_result)

RAGAS Framework : Context Precision
- How relevant are the retrieved documents to the query?
- Normalized to (0,1) -> 1 = highly relevant

In [ ]:
from ragas. metrics import context_precision

llm = ChatOpenAI(model="gpt-40-mini", api_key="...")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", api_key="...")

context_precision_chain = EvaluatorChain(
  metric=context_precision,
  llm=llm,
  embeddings=embeddings
)

eval_result = context_precision_chain({
"question": "How does the RAG model improve question answering with large language models?",
"ground_truth": "The RAG model improves question answering by combining the retrieval of...",
"contexts": [
  "The RAG model integrates document retrieval with LLMs by first retrieving...",
  "By incorporating retrieval mechanisms, RAG leverages external knowledge sources..."
]
})

print(f"Context Precision: {eval_result[ 'context_precision']}")